In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

import cupy as cp
import cupyx
import cupyx.scipy.sparse.linalg
import numpy as np
import time
import timeit
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
L = 30
N_x = 10000
N_t = 1000
x = cp.linspace(-L, L, N_x)
t = cp.linspace(0, 10, N_t)
dt = t[1] - t[0]
dx = x[1] - x[0]

p_0 = 10
x_0 = 0
sigma_x = 1
psi_0 = cp.exp(-(x-x_0)**2 / (2 * sigma_x ** 2)) * cp.exp(+1j*p_0*x)
norm = cp.sqrt(cp.sum(cp.absolute(psi_0) ** 2) * dx)
psi_0 = psi_0 / norm

psi = cp.zeros((N_x, N_t), dtype=cp.complex64)
psi[:, 0] = psi_0


V = x ** 2 / 2

In [3]:
def time_step(psi_0, LHS, RHS):
  psi_0 = cupyx.scipy.sparse.linalg.spsolve(LHS, RHS.dot(psi_0))
  norm = cp.sum(cp.absolute(psi_0) ** 2) * dx
  psi_0 = psi_0 / cp.sqrt(norm)

In [ ]:
L = 30
N_x = [10 ** i for i in range(2, 7)]
N_t = 1000
time_gpu = []
for n in tqdm(N_x):
  x = cp.linspace(-L, L, n)
  t = cp.linspace(0, 10, N_t)
  dt = t[1] - t[0]
  dx = x[1] - x[0]
  psi_0 = cp.exp(-(x-x_0)**2 / (2 * sigma_x ** 2)) * cp.exp(+1j*p_0*x)
  norm = cp.sqrt(cp.sum(cp.absolute(psi_0) ** 2) * dx)
  psi_0 = psi_0 / norm
  gamma = (1j / 4) * (dt / dx ** 2)
  V = x ** 2 / 2
  RHS = cupyx.scipy.sparse.diags([[gamma]*(n-1), 1-2*gamma-1j*dt*V/2, [gamma]*(n-1)], offsets=[-1, 0, 1], shape=(n, n)).tocsr()
  LHS = cupyx.scipy.sparse.diags([[-gamma]*(n-1), 1+2*gamma+1j*dt*V/2, [-gamma]*(n-1)], offsets=[-1, 0, 1], shape=(n, n)).tocsr()
  time0 = timeit.timeit(lambda: time_step(psi_0, LHS, RHS), number=100)
  time_gpu.append(time0)

In [5]:
time_gpu

[0.5025563629997123, 2.207509797000057, 21.499866512000153, 217.3448648170006]

In [10]:
def time_step(psi_0, LHS, RHS):
  psi_0 = cp.linalg.solve(LHS, RHS.dot(psi_0))
  norm = cp.sum(cp.absolute(psi_0) ** 2) * dx
  psi_0 = psi_0 / cp.sqrt(norm)


L = 30
N_x = [10 ** i for i in range(2, 5)]
N_t = 1000
time_gpu = []
for n in tqdm(N_x):
  x = cp.linspace(-L, L, n)
  t = cp.linspace(0, 10, N_t)
  dt = t[1] - t[0]
  dx = x[1] - x[0]
  psi_0 = cp.exp(-(x-x_0)**2 / (2 * sigma_x ** 2)) * cp.exp(+1j*p_0*x)
  norm = cp.sqrt(cp.sum(cp.absolute(psi_0) ** 2) * dx)
  psi_0 = psi_0 / norm
  gamma = (1j / 4) * (dt / dx ** 2)
  V = x ** 2 / 2
  RHS = cp.diag(1-2*gamma-1j*dt*V/2) + cp.diag(cp.array([gamma]*(n-1)), 1) + cp.diag(cp.array([gamma]*(n-1)), -1)
  LHS = cp.diag(1+2*gamma+1j*dt*V/2) + cp.diag(cp.array([-gamma]*(n-1)), 1) + cp.diag(cp.array([-gamma]*(n-1)), -1)
  time0 = timeit.timeit(lambda: time_step(psi_0, LHS, RHS), number=100)
  time_gpu.append(time0)

100%|██████████| 3/3 [18:16<00:00, 365.60s/it]


In [11]:
time_gpu

[0.06827249900015886, 1.9226194129996657, 1094.386678797]